# CPU Exact vs ANN Quality and Performance Suite

This notebook reuses the same synthetic datasets/scenarios from `cpu_ann_vs_exact.ipynb`, but compares **Rust exact** (`approx=False`) against **Rust ANN** (`approx=True`) directly.

Primary metrics include speedup (`ann_speedup_vs_exact`) and quality (`top-k overlap`, `top-1 match rate`, `nDCG@k`, and similarity deltas).


In [1]:
from dataclasses import dataclass
from time import perf_counter
from typing import Callable

import numpy as np
import pandas as pd
from nndex import NNdex

EPSILON = 1e-12

MatrixBuilder = Callable[[np.random.Generator, int, int], np.ndarray]


@dataclass(frozen=True)
class Scenario:
    name: str
    rows: int
    dims: int
    query_rows: int
    k: int
    tolerance: float
    builder: MatrixBuilder


def normalize_rows(matrix: np.ndarray) -> np.ndarray:
    norms = np.linalg.norm(matrix, axis=1, keepdims=True).astype(np.float32)
    return (matrix / np.maximum(norms, np.float32(EPSILON))).astype(np.float32, copy=False)


def python_cosine_topk_batch(
    normalized_matrix: np.ndarray,
    queries: np.ndarray,
    k: int,
) -> tuple[np.ndarray, np.ndarray]:
    q_norm = normalize_rows(queries)
    sims = q_norm @ normalized_matrix.T

    candidate_idx = np.argpartition(sims, -k, axis=1)[:, -k:]
    row_ids = np.arange(sims.shape[0])[:, None]
    candidate_scores = sims[row_ids, candidate_idx]
    order = np.argsort(candidate_scores, axis=1)[:, ::-1]

    topk_idx = candidate_idx[row_ids, order].astype(np.int64, copy=False)
    topk_scores = candidate_scores[row_ids, order].astype(np.float32, copy=False)
    return topk_idx, topk_scores


def python_cosine_topk_single(
    normalized_matrix: np.ndarray,
    query: np.ndarray,
    k: int,
) -> tuple[np.ndarray, np.ndarray]:
    """Single-query cosine top-k via matrix-vector product (BLAS sgemv path)."""
    norm = np.float32(max(float(np.linalg.norm(query)), EPSILON))
    q_norm = (query / norm).astype(np.float32, copy=False)
    sims = normalized_matrix @ q_norm

    candidate_idx = np.argpartition(sims, -k)[-k:]
    candidate_scores = sims[candidate_idx]
    order = np.argsort(candidate_scores)[::-1]

    topk_idx = candidate_idx[order].astype(np.int64, copy=False)
    topk_scores = candidate_scores[order].astype(np.float32, copy=False)
    return topk_idx, topk_scores


def topk_overlap_fraction(rust_idx: np.ndarray, python_idx: np.ndarray) -> float:
    overlaps: list[float] = []
    for rust_row, py_row in zip(rust_idx, python_idx):
        overlaps.append(len(set(rust_row.tolist()) & set(py_row.tolist())) / rust_row.size)
    return float(np.mean(overlaps))


def median_ms(func: Callable[[], object], repeats: int) -> float:
    timings = np.empty(repeats, dtype=np.float64)
    for i in range(repeats):
        start = perf_counter()
        _ = func()
        timings[i] = (perf_counter() - start) * 1_000.0
    return float(np.median(timings))


def build_queries(matrix: np.ndarray, rng: np.random.Generator, query_rows: int) -> np.ndarray:
    row_ids = rng.integers(0, matrix.shape[0], size=query_rows)
    jitter = rng.normal(loc=0.0, scale=0.02, size=(query_rows, matrix.shape[1]))
    return (matrix[row_ids] + jitter.astype(np.float32)).astype(np.float32, copy=False)


# Unusual: multiplicative/nonlinear features that stress interaction sensitivity.
def interaction_effects_matrix(rng: np.random.Generator, rows: int, dims: int) -> np.ndarray:
    core = rng.normal(size=(rows, 8)).astype(np.float32)
    interactions = np.column_stack(
        [
            core[:, 0] * core[:, 1],
            core[:, 0] * core[:, 2],
            core[:, 1] * core[:, 3],
            core[:, 4] * core[:, 5],
            np.sin(core[:, 6]),
            np.cos(core[:, 7]),
            core[:, 2] ** 2,
            core[:, 3] ** 2,
        ]
    ).astype(np.float32)
    features = np.concatenate([core, interactions], axis=1)
    if features.shape[1] >= dims:
        return features[:, :dims].astype(np.float32, copy=False)
    pad = rng.normal(0.0, 0.05, size=(rows, dims - features.shape[1])).astype(np.float32)
    return np.concatenate([features, pad], axis=1)


# Unusual: vectors constrained to noisy radius shells; tests angular ranking robustness.
def concentric_shells_matrix(rng: np.random.Generator, rows: int, dims: int) -> np.ndarray:
    directions = rng.normal(size=(rows, dims)).astype(np.float32)
    norms = np.linalg.norm(directions, axis=1, keepdims=True).astype(np.float32)
    radii = rng.choice(np.array([0.55, 1.25, 2.1, 3.4], dtype=np.float32), size=rows)
    noise = rng.normal(loc=0.0, scale=0.03, size=(rows, dims)).astype(np.float32)
    shells = directions / np.maximum(norms, np.float32(EPSILON))
    return (shells * radii[:, None] + noise).astype(np.float32, copy=False)


# Unusual: low-rank signal plus noise; tests behavior on compressible embeddings.
def low_rank_plus_noise_matrix(rng: np.random.Generator, rows: int, dims: int) -> np.ndarray:
    rank = min(12, max(4, dims // 8))
    latent = rng.normal(size=(rows, rank)).astype(np.float32)
    loadings = rng.normal(size=(rank, dims)).astype(np.float32)
    structured = latent @ loadings
    noise = rng.normal(loc=0.0, scale=0.04, size=(rows, dims)).astype(np.float32)
    return (structured + noise).astype(np.float32, copy=False)


# Unusual: mostly sparse spikes with periodic drift; tests sparse/high-dim retrieval.
def sparse_spikes_matrix(rng: np.random.Generator, rows: int, dims: int) -> np.ndarray:
    matrix = rng.normal(loc=0.0, scale=0.015, size=(rows, dims)).astype(np.float32)
    spikes = max(2, dims // 32)
    row_ids = np.repeat(np.arange(rows), spikes)
    col_ids = rng.integers(0, dims, size=rows * spikes)
    values = rng.normal(loc=0.0, scale=1.2, size=rows * spikes).astype(np.float32)
    matrix[row_ids, col_ids] += values
    matrix += np.sin(np.arange(dims, dtype=np.float32) / 9.0)[None, :] * np.float32(0.08)
    return matrix


# Unusual: curved manifold projected into feature space; tests non-Euclidean structure.
def curve_manifold_matrix(rng: np.random.Generator, rows: int, dims: int) -> np.ndarray:
    t = rng.uniform(0.0, 6.0 * np.pi, size=(rows, 1)).astype(np.float32)
    manifold = np.concatenate(
        [np.sin(t), np.cos(t), np.sin(2.0 * t), np.cos(2.0 * t), t / np.float32(6.0 * np.pi)],
        axis=1,
    ).astype(np.float32)
    projection = rng.normal(size=(manifold.shape[1], dims)).astype(np.float32)
    embedded = manifold @ projection
    noise = rng.normal(loc=0.0, scale=0.025, size=(rows, dims)).astype(np.float32)
    return (embedded + noise).astype(np.float32, copy=False)


# Unusual: block-correlated dimensions; tests ranking under covariance clusters.
def block_correlated_matrix(rng: np.random.Generator, rows: int, dims: int) -> np.ndarray:
    blocks = max(3, min(12, dims // 8))
    block_width = (dims + blocks - 1) // blocks
    latent = rng.normal(size=(rows, blocks)).astype(np.float32)
    matrix = np.empty((rows, dims), dtype=np.float32)
    for block_idx in range(blocks):
        start = block_idx * block_width
        end = min(dims, start + block_width)
        if start >= end:
            break
        jitter = rng.normal(loc=0.0, scale=0.06, size=(rows, end - start)).astype(np.float32)
        matrix[:, start:end] = latent[:, [block_idx]] + jitter
    return matrix


# Unusual: chirped sinusoid feature banks; tests structured periodic variation.
def chirp_wave_bank_matrix(rng: np.random.Generator, rows: int, dims: int) -> np.ndarray:
    x = np.linspace(0.0, 1.0, num=dims, dtype=np.float32)[None, :]
    base_freq = rng.uniform(1.5, 16.0, size=(rows, 1)).astype(np.float32)
    chirp = rng.uniform(0.5, 5.0, size=(rows, 1)).astype(np.float32)
    phase = rng.uniform(0.0, 2.0 * np.pi, size=(rows, 1)).astype(np.float32)
    waves = np.sin((base_freq * x + chirp * x * x) * np.float32(2.0 * np.pi) + phase)
    harmonics = np.cos((base_freq * np.float32(0.5) * x) * np.float32(2.0 * np.pi) - phase)
    noise = rng.normal(loc=0.0, scale=0.03, size=(rows, dims)).astype(np.float32)
    return (waves + np.float32(0.35) * harmonics + noise).astype(np.float32, copy=False)


# Unusual: piecewise trends with jumps; tests retrieval on nonstationary vectors.
def piecewise_jump_trend_matrix(rng: np.random.Generator, rows: int, dims: int) -> np.ndarray:
    segments = 5
    boundaries = np.linspace(0, dims, num=segments + 1, dtype=np.int64)
    matrix = rng.normal(loc=0.0, scale=0.08, size=(rows, dims)).astype(np.float32)
    slopes = rng.normal(loc=0.0, scale=0.9, size=(rows, segments)).astype(np.float32)
    for segment_idx in range(segments):
        start = int(boundaries[segment_idx])
        end = int(boundaries[segment_idx + 1])
        if start >= end:
            continue
        ramp = np.linspace(0.0, 1.0, num=end - start, dtype=np.float32)[None, :]
        matrix[:, start:end] += slopes[:, [segment_idx]] * ramp
        if segment_idx + 1 < segments:
            jump = rng.normal(loc=0.0, scale=0.35, size=(rows, 1)).astype(np.float32)
            matrix[:, end:] += jump
    seasonal = np.sin(np.arange(dims, dtype=np.float32) / np.float32(7.0))[None, :]
    return (matrix + np.float32(0.05) * seasonal).astype(np.float32, copy=False)


# Unusual: mixed positive/negative topic prototypes; tests signed semantic superposition.
def signed_topic_mixture_matrix(rng: np.random.Generator, rows: int, dims: int) -> np.ndarray:
    topic_count = 20
    topics = rng.normal(size=(topic_count, dims)).astype(np.float32)
    topic_ids_a = rng.integers(0, topic_count, size=rows)
    topic_ids_b = rng.integers(0, topic_count, size=rows)
    blend = rng.uniform(-1.0, 1.0, size=(rows, 1)).astype(np.float32)
    matrix = topics[topic_ids_a] * blend + topics[topic_ids_b] * (np.float32(1.0) - blend)
    gates = rng.choice(np.array([-1.0, 1.0], dtype=np.float32), size=(rows, 1))
    noise = rng.normal(loc=0.0, scale=0.02, size=(rows, dims)).astype(np.float32)
    return (matrix * gates + noise).astype(np.float32, copy=False)


# Unusual: binary hashes projected to dense space; tests discontinuous-to-dense embeddings.
def binary_hash_projection_matrix(rng: np.random.Generator, rows: int, dims: int) -> np.ndarray:
    bit_count = 64
    bits = rng.integers(0, 2, size=(rows, bit_count), dtype=np.int8).astype(np.float32)
    signed_bits = bits * np.float32(2.0) - np.float32(1.0)
    projection = rng.normal(loc=0.0, scale=0.2, size=(bit_count, dims)).astype(np.float32)
    dense = signed_bits @ projection
    trend = rng.normal(loc=0.0, scale=0.04, size=(rows, 1)).astype(np.float32)
    axis = np.linspace(-1.0, 1.0, num=dims, dtype=np.float32)[None, :]
    return (dense + trend * axis).astype(np.float32, copy=False)


BATCH_SCENARIOS = (
    Scenario('interaction_effects', 12_345, 33, 128, 15, 1e-4, interaction_effects_matrix),
    Scenario('concentric_shells', 9_876, 129, 128, 15, 1e-4, concentric_shells_matrix),
    Scenario('low_rank_plus_noise', 15_555, 192, 128, 15, 1e-4, low_rank_plus_noise_matrix),
    Scenario('sparse_spikes', 18_181, 640, 96, 15, 2e-4, sparse_spikes_matrix),
    Scenario('curve_manifold_projection', 7_777, 11, 128, 15, 1e-4, curve_manifold_matrix),
    Scenario('block_correlated_large', 50_000, 48, 160, 20, 1e-4, block_correlated_matrix),
    Scenario('chirp_wave_bank_large', 55_555, 96, 160, 20, 1e-4, chirp_wave_bank_matrix),
    Scenario('piecewise_jump_trend_large', 62_500, 257, 128, 20, 1e-4, piecewise_jump_trend_matrix),
    Scenario('signed_topic_mixture_large', 70_000, 384, 96, 20, 2e-4, signed_topic_mixture_matrix),
    Scenario('binary_hash_projection_large', 80_000, 768, 64, 20, 2e-4, binary_hash_projection_matrix),
)

SINGLE_QUERY_SCENARIOS = (
    Scenario('interaction_effects', 12_345, 33, 1, 15, 1e-4, interaction_effects_matrix),
    Scenario('concentric_shells', 9_876, 129, 1, 15, 1e-4, concentric_shells_matrix),
    Scenario('low_rank_plus_noise', 15_555, 192, 1, 15, 1e-4, low_rank_plus_noise_matrix),
    Scenario('sparse_spikes', 18_181, 640, 1, 15, 2e-4, sparse_spikes_matrix),
    Scenario('curve_manifold_projection', 7_777, 11, 1, 15, 1e-4, curve_manifold_matrix),
    Scenario('block_correlated_large', 50_000, 48, 1, 20, 1e-4, block_correlated_matrix),
    Scenario('chirp_wave_bank_large', 55_555, 96, 1, 20, 1e-4, chirp_wave_bank_matrix),
    Scenario('piecewise_jump_trend_large', 62_500, 257, 1, 20, 1e-4, piecewise_jump_trend_matrix),
    Scenario('signed_topic_mixture_large', 70_000, 384, 1, 20, 2e-4, signed_topic_mixture_matrix),
    Scenario('binary_hash_projection_large', 80_000, 768, 1, 20, 2e-4, binary_hash_projection_matrix),
)


def _dcg(scores: np.ndarray) -> float:
    discounts = np.log2(np.arange(2, scores.size + 2, dtype=np.float64))
    return float(np.sum(scores.astype(np.float64) / discounts))


def _query_quality_metrics(
    exact_idx: np.ndarray,
    exact_scores: np.ndarray,
    ann_idx: np.ndarray,
    ann_scores: np.ndarray,
) -> dict[str, float]:
    exact_set = set(exact_idx.tolist())
    ann_set = set(ann_idx.tolist())
    overlap = len(exact_set & ann_set) / exact_idx.size

    exact_map = {int(idx): float(score) for idx, score in zip(exact_idx.tolist(), exact_scores.tolist())}
    ann_map = {int(idx): float(score) for idx, score in zip(ann_idx.tolist(), ann_scores.tolist())}
    shared = exact_set & ann_set

    overlap_score_mae = (
        float(np.mean([abs(exact_map[i] - ann_map[i]) for i in shared]))
        if shared
        else float('nan')
    )

    ann_relevance = np.array([exact_map.get(int(i), 0.0) for i in ann_idx], dtype=np.float32)
    ideal_relevance = exact_scores.astype(np.float32, copy=False)
    dcg_ideal = max(_dcg(ideal_relevance), EPSILON)
    ndcg = _dcg(ann_relevance) / dcg_ideal

    return {
        'topk_overlap': overlap,
        'top1_match': float(exact_idx[0] == ann_idx[0]),
        'ndcg_at_k': ndcg,
        'rank_position_score_mae': float(np.mean(np.abs(exact_scores - ann_scores))),
        'overlap_score_mae': overlap_score_mae,
        'top1_similarity_abs_delta': float(abs(float(exact_scores[0]) - float(ann_scores[0]))),
        'exact_topk_mean_similarity': float(np.mean(exact_scores)),
        'ann_topk_mean_similarity': float(np.mean(ann_scores)),
    }


def _aggregate_quality(
    exact_idx: np.ndarray,
    exact_scores: np.ndarray,
    ann_idx: np.ndarray,
    ann_scores: np.ndarray,
) -> dict[str, float]:
    if exact_idx.ndim == 1:
        exact_idx = exact_idx[None, :]
        exact_scores = exact_scores[None, :]
        ann_idx = ann_idx[None, :]
        ann_scores = ann_scores[None, :]

    per_query = [
        _query_quality_metrics(exact_idx[i], exact_scores[i], ann_idx[i], ann_scores[i])
        for i in range(exact_idx.shape[0])
    ]

    overlap_values = np.array([row['topk_overlap'] for row in per_query], dtype=np.float64)
    top1_values = np.array([row['top1_match'] for row in per_query], dtype=np.float64)
    ndcg_values = np.array([row['ndcg_at_k'] for row in per_query], dtype=np.float64)
    rank_mae_values = np.array([row['rank_position_score_mae'] for row in per_query], dtype=np.float64)
    overlap_mae_values = np.array([row['overlap_score_mae'] for row in per_query], dtype=np.float64)
    top1_delta_values = np.array([row['top1_similarity_abs_delta'] for row in per_query], dtype=np.float64)
    exact_mean_values = np.array([row['exact_topk_mean_similarity'] for row in per_query], dtype=np.float64)
    ann_mean_values = np.array([row['ann_topk_mean_similarity'] for row in per_query], dtype=np.float64)

    return {
        'topk_overlap_mean': float(np.mean(overlap_values)),
        'topk_overlap_min': float(np.min(overlap_values)),
        'top1_match_rate': float(np.mean(top1_values)),
        'ndcg_at_k_mean': float(np.mean(ndcg_values)),
        'rank_position_score_mae_mean': float(np.mean(rank_mae_values)),
        'overlap_score_mae_mean': float(np.nanmean(overlap_mae_values)),
        'top1_similarity_abs_delta_mean': float(np.mean(top1_delta_values)),
        'exact_topk_mean_similarity': float(np.mean(exact_mean_values)),
        'ann_topk_mean_similarity': float(np.mean(ann_mean_values)),
        'ann_minus_exact_similarity_mean': float(np.mean(ann_mean_values - exact_mean_values)),
    }


def run_ann_vs_exact_batch_suite(repeats: int = 7, seed: int = 20260224) -> pd.DataFrame:
    rng = np.random.default_rng(seed)
    rows: list[dict[str, float | int | str]] = []

    for s in BATCH_SCENARIOS:
        matrix = s.builder(rng, s.rows, s.dims)
        queries = build_queries(matrix, rng, s.query_rows)

        exact_index = NNdex(
            matrix,
            normalized=False,
            approx=False,
            backend='cpu',
            enable_cache=False,
        )
        ann_index = NNdex(
            matrix,
            normalized=False,
            approx=True,
            backend='cpu',
            enable_cache=False,
        )

        exact_idx, exact_scores = exact_index.search(queries, k=s.k)
        ann_idx, ann_scores = ann_index.search(queries, k=s.k)

        quality = _aggregate_quality(exact_idx, exact_scores, ann_idx, ann_scores)
        exact_ms = median_ms(lambda: exact_index.search(queries, k=s.k), repeats)
        ann_ms = median_ms(lambda: ann_index.search(queries, k=s.k), repeats)

        rows.append(
            {
                'scenario': s.name,
                'rows': s.rows,
                'dims': s.dims,
                'query_rows': s.query_rows,
                'k': s.k,
                'exact_median_ms': exact_ms,
                'ann_median_ms': ann_ms,
                'ann_speedup_vs_exact': exact_ms / ann_ms,
                **quality,
            }
        )

    return pd.DataFrame(rows).sort_values('ann_speedup_vs_exact', ascending=False)


def run_ann_vs_exact_single_query_suite(repeats: int = 31, seed: int = 20260224) -> pd.DataFrame:
    rng = np.random.default_rng(seed)
    query_bank_size = 32
    rows: list[dict[str, float | int | str]] = []

    for s in SINGLE_QUERY_SCENARIOS:
        matrix = s.builder(rng, s.rows, s.dims)
        query_bank = build_queries(matrix, rng, query_bank_size)

        exact_index = NNdex(
            matrix,
            normalized=False,
            approx=False,
            backend='cpu',
            enable_cache=False,
        )
        ann_index = NNdex(
            matrix,
            normalized=False,
            approx=True,
            backend='cpu',
            enable_cache=False,
        )

        exact_idx_rows: list[np.ndarray] = []
        exact_score_rows: list[np.ndarray] = []
        ann_idx_rows: list[np.ndarray] = []
        ann_score_rows: list[np.ndarray] = []

        for query in query_bank:
            exact_idx, exact_scores = exact_index.search(query, k=s.k)
            ann_idx, ann_scores = ann_index.search(query, k=s.k)
            exact_idx_rows.append(exact_idx)
            exact_score_rows.append(exact_scores)
            ann_idx_rows.append(ann_idx)
            ann_score_rows.append(ann_scores)

        quality = _aggregate_quality(
            np.stack(exact_idx_rows, axis=0),
            np.stack(exact_score_rows, axis=0),
            np.stack(ann_idx_rows, axis=0),
            np.stack(ann_score_rows, axis=0),
        )

        cycle_idx = 0

        def exact_single() -> object:
            nonlocal cycle_idx
            q = query_bank[cycle_idx % query_bank_size]
            cycle_idx += 1
            return exact_index.search(q, k=s.k)

        cycle_idx_ann = 0

        def ann_single() -> object:
            nonlocal cycle_idx_ann
            q = query_bank[cycle_idx_ann % query_bank_size]
            cycle_idx_ann += 1
            return ann_index.search(q, k=s.k)

        exact_ms = median_ms(exact_single, repeats)
        ann_ms = median_ms(ann_single, repeats)

        rows.append(
            {
                'scenario': s.name,
                'rows': s.rows,
                'dims': s.dims,
                'query_rows': 1,
                'k': s.k,
                'exact_median_ms': exact_ms,
                'ann_median_ms': ann_ms,
                'ann_speedup_vs_exact': exact_ms / ann_ms,
                **quality,
            }
        )

    return pd.DataFrame(rows).sort_values('ann_speedup_vs_exact', ascending=False)



## Batch Query Comparison (query_rows > 1)

Evaluates exact vs ANN for each batch scenario using the same generated matrix and query batch.


In [2]:
batch_ann_df = run_ann_vs_exact_batch_suite(repeats=7)
batch_ann_df.round(
    {
        'exact_median_ms': 3,
        'ann_median_ms': 3,
        'ann_speedup_vs_exact': 3,
        'topk_overlap_mean': 4,
        'topk_overlap_min': 4,
        'top1_match_rate': 4,
        'ndcg_at_k_mean': 4,
        'rank_position_score_mae_mean': 8,
        'overlap_score_mae_mean': 8,
        'top1_similarity_abs_delta_mean': 8,
        'exact_topk_mean_similarity': 6,
        'ann_topk_mean_similarity': 6,
        'ann_minus_exact_similarity_mean': 8,
    }
).reset_index(drop=True)


scenario   rows  dims  query_rows   k  exact_median_ms  \
0           low_rank_plus_noise  15555   192         128  15            7.904   
1    signed_topic_mixture_large  70000   384          96  20           57.262   
2        block_correlated_large  50000    48         160  20            9.691   
3  binary_hash_projection_large  80000   768          64  20           90.699   
4         chirp_wave_bank_large  55555    96         160  20           19.185   
5    piecewise_jump_trend_large  62500   257         128  20           42.393   
6             concentric_shells   9876   129         128  15            3.741   
7                 sparse_spikes  18181   640          96  15           23.856   
8           interaction_effects  12345    33         128  15            1.754   
9     curve_manifold_projection   7777    11         128  15            0.776   

   ann_median_ms  ann_speedup_vs_exact  topk_overlap_mean  topk_overlap_min  \
0          0.590                13.386             0.7547            0.3333   
1          4.615                12.408             0.9995            0.9500   
2          0.803                12.069             1.0000            1.0000   
3          8.770                10.342             0.5305            0.2500   
4          1.978                 9.699             1.0000            1.0000   
5          4.742                 8.939             0.9992            0.9500   
6          0.467                 8.018             0.3266            0.0667   
7          3.337                 7.149             0.4451            0.1333   
8          0.266                 6.582             0.9484            0.6000   
9          0.168                 4.622             1.0000            1.0000   

   top1_match_rate  ndcg_at_k_mean  rank_position_score_mae_mean  \
0              1.0          0.8413                  1.027844e-02   
1              1.0          0.9997                  1.400000e-07   
2              1.0          1.0000                  7.000000e-08   
3              1.0          0.7109                  1.905252e-02   
4              1.0          1.0000                  1.000000e-07   
5              1.0          0.9995                  2.616000e-05   
6              1.0          0.6368                  3.161175e-02   
7              1.0          0.6908                  3.156624e-02   
8              1.0          0.9675                  1.305910e-03   
9              1.0          1.0000                  4.000000e-08   

   overlap_score_mae_mean  top1_similarity_abs_delta_mean  \
0            1.300000e-07                    1.600000e-07   
1            1.400000e-07                    1.300000e-07   
2            7.000000e-08                    7.000000e-08   
3            7.000000e-08                    1.200000e-07   
4            1.000000e-07                    1.000000e-07   
5            1.400000e-07                    1.500000e-07   
6            5.000000e-08                    1.200000e-07   
7            7.000000e-08                    1.400000e-07   
8            7.000000e-08                    7.000000e-08   
9            4.000000e-08                    4.000000e-08   

   exact_topk_mean_similarity  ann_topk_mean_similarity  \
0                    0.827288                  0.817009   
1                    0.996088                  0.996088   
2                    0.965475                  0.965475   
3                    0.499478                  0.480425   
4                    0.952824                  0.952824   
5                    0.942129                  0.942103   
6                    0.328120                  0.296508   
7                    0.374410                  0.342843   
8                    0.886646                  0.885340   
9                    0.999334                  0.999334   

   ann_minus_exact_similarity_mean  
0                        -0.010278  
1                        -0.000000  
2                        -0.000000  
3                        -0.019053  
4    

## Single Query Comparison (query_rows = 1)

Evaluates exact vs ANN over a 32-query bank per scenario to stabilize single-query quality and timing.


In [3]:
single_ann_df = run_ann_vs_exact_single_query_suite(repeats=31)
single_ann_df.round(
    {
        'exact_median_ms': 3,
        'ann_median_ms': 3,
        'ann_speedup_vs_exact': 3,
        'topk_overlap_mean': 4,
        'topk_overlap_min': 4,
        'top1_match_rate': 4,
        'ndcg_at_k_mean': 4,
        'rank_position_score_mae_mean': 8,
        'overlap_score_mae_mean': 8,
        'top1_similarity_abs_delta_mean': 8,
        'exact_topk_mean_similarity': 6,
        'ann_topk_mean_similarity': 6,
        'ann_minus_exact_similarity_mean': 8,
    }
).reset_index(drop=True)


scenario   rows  dims  query_rows   k  exact_median_ms  \
0    signed_topic_mixture_large  70000   384           1  20            1.062   
1     curve_manifold_projection   7777    11           1  15            0.029   
2           interaction_effects  12345    33           1  15            0.048   
3           low_rank_plus_noise  15555   192           1  15            0.117   
4  binary_hash_projection_large  80000   768           1  20            2.270   
5    piecewise_jump_trend_large  62500   257           1  20            0.725   
6             concentric_shells   9876   129           1  15            0.077   
7         chirp_wave_bank_large  55555    96           1  20            0.250   
8        block_correlated_large  50000    48           1  20            0.119   
9                 sparse_spikes  18181   640           1  15            0.479   

   ann_median_ms  ann_speedup_vs_exact  topk_overlap_mean  topk_overlap_min  \
0          0.151                 7.035             1.0000            1.0000   
1          0.005                 6.357             1.0000            1.0000   
2          0.007                 6.350             0.9438            0.7333   
3          0.019                 6.078             0.7979            0.4667   
4          0.392                 5.790             0.5641            0.3500   
5          0.132                 5.470             1.0000            1.0000   
6          0.015                 5.094             0.3708            0.1333   
7          0.053                 4.728             1.0000            1.0000   
8          0.028                 4.325             1.0000            1.0000   
9          0.242                 1.981             0.4771            0.1333   

   top1_match_rate  ndcg_at_k_mean  rank_position_score_mae_mean  \
0              1.0          1.0000                  7.000000e-08   
1              1.0          1.0000                  3.000000e-08   
2              1.0          0.9649                  1.301530e-03   
3              1.0          0.8691                  7.820350e-03   
4              1.0          0.7363                  1.738602e-02   
5              1.0          1.0000                  5.000000e-08   
6              1.0          0.6647                  2.970365e-02   
7              1.0          1.0000                  4.000000e-08   
8              1.0          1.0000                  3.000000e-08   
9              1.0          0.7152                  2.583278e-02   

   overlap_score_mae_mean  top1_similarity_abs_delta_mean  \
0            7.000000e-08                    7.000000e-08   
1            3.000000e-08                    3.000000e-08   
2            4.000000e-08                    4.000000e-08   
3            4.000000e-08                    5.000000e-08   
4            5.000000e-08                    9.000000e-08   
5            5.000000e-08                    6.000000e-08   
6            2.000000e-08                    5.000000e-08   
7            4.000000e-08                    5.000000e-08   
8            3.000000e-08                    5.000000e-08   
9            4.000000e-08                    7.000000e-08   

   exact_topk_mean_similarity  ann_topk_mean_similarity  \
0                    0.996731                  0.996731   
1                    0.999280                  0.999280   
2                    0.877591                  0.876289   
3                    0.833631                  0.825811   
4                    0.500325                  0.482939   
5                    0.945259                  0.945259   
6                    0.328320                  0.298616   
7                    0.952150                  0.952150   
8                    0.964470                  0.964470   
9                    0.369755                  0.343923   

   ann_minus_exact_similarity_mean  
0                     2.000000e-08  
1                     0.000000e+00  
2                    -1.301520e-03  
3                    -7.820320e-03  
4    